In [1]:
# Nikhef Modulation Experiment Simulation
# Simulates data of radioactive decay, assuming exponential decay function, with Gaussian statistical uncertainty 
# (intrinsic to radioactive decay) and extra noise from errors in measurement should still be added. 
# A modulation can be manually added.


# Input: 
#    Days_Measured = days measured
#    Num_sample_points = number of sample points, eg: 1 sample per minute => N = 1*60*24*D
#    tau_in_Yr = half life time in years
#    A_0 = activity of radioactive source at t = 0
#    freq, ampl, phaseDg: frequency (1 modulation per day -> 1/(24*60*60)), amplitude and phase of signal (in degrees)
#    sigma_Noise = standard deviation of uncertainty in measurement on signal
#    guess_hLife_time = start value for half life time, set to 1 year, since 60Co, 137Cs and 44Ti have half life times in order of years

# DON'T FORGET TO CHANGE THE #DAYS IN THE NP.SAVETXT-FILENAME!!! 

import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.optimize as optimize
from scipy.optimize import curve_fit
%matplotlib inline

#simulation of exponential decay
Days_Measured = 50. # days measured
T = 60. #sample spacing [sec] = total time measured (in seconds) /number of sample points
Num_sample_points = (60*60*24.0*Days_Measured)/T # number of sample points (if 1 per minute: 60*24 in 1 day & 1 per 6 min: 10*24 in 1 day)
                                                 # Num_sample_points should preferably be 2^n with n = integer, this to make the FFT work better
day_in_mintes = 24*60 # number of minutes in a day
day_in_seconds = 24*60*60 # number of seconds in a day

tau_in_Yr = 5.27 #half life time in years
tau_in_Sec = tau_in_Yr*365*24*60*60 #half life time in seconds

A0 = 37 #start activity in Bq

x = np.linspace(0, T*Num_sample_points, Num_sample_points)

def exp(x, A_0, half_life_time): # A_0=activity at t=0 in Bq, and half life time in seconds 
    return A_0*np.exp(-x*np.log(2)/half_life_time)

#exponential radioactive decay
def exp_decay(x):
    return exp(x,A0,tau_in_Sec)

In [2]:
# Define all needed functions

def mod(x,ampl,freq1,pha):
    return ampl*np.cos(2*np.pi*freq1*x-pha)

# Radioactive decay + modulation:
def exp_decay_and_Modulation(x):
    return exp_decay(x)+mod(x,ampl_input, freq_input,phase_input)

# add Gaussian noise proportoinal to the error in our measurements (to make it a realistic signal)
sigma_Noise_external = np.sqrt(T*exp_decay(x))/(T)  # sigma_A = sigma_N/T = sqrt(N)/T = sqrt(AT)/T

def external_Noise(x):
    return np.random.normal(0,sigma_Noise_external,len(x))

def exp_decay_mod_all_noise(x):
    return exp_decay_and_Modulation(x)+external_Noise(x)

# fit exponent to simulated radioactive decay and find Chi-squared
def fit_signal(signal, guess, x):
    popt, pcov = curve_fit(exp, x, signal, guess, maxfev=200)
    perr = np.sqrt(np.diag(pcov))
    Fitted_exp_decay = exp(x, *popt)
    return Fitted_exp_decay

# Define the residuals als the data mnis the fitted exponent
def residual(x, exponent_and_modulation_and_noise, fit):#, plot_function = False):
    residual = (exponent_and_modulation_and_noise) - fit
    return residual

# find modulation frequency using a FFT
def FFT_residual (residuals):
    return np.fft.rfft(residuals)

def FFTfreq (residuals):
    return np.fft.rfftfreq(len(residuals))


In [3]:
range_ampl = np.arange(0., 0.001, 0.00005)
range_run = np.arange(0., 300, 1.)

ampl_input_array = []
frequency_FFT_varying_ampl_array = []
stdv_frequency_FFT_varying_ampl_array = []
FFT_found_ampl_varying_ampl_array = []
stdv_FFT_found_ampl_varying_ampl_array = []
FFT_found_phase_varying_ampl_array = []
stdv_FFT_found_phase_varying_ampl_array = []


for ampl_run in range_ampl:

    # artificially added modulation:
    freq_input = 1./(24*60*60.0) #frequency of the signal (mod/sec): 1 modulation per day -> 1/(24*60*60)
    ampl_input = ampl_run*A0 #amplitude of the signal, changes in activity of ~0.1% should be detectable
    ampl_input_array.append(ampl_input) 
    phase_in_Degrees = 40.
    phase_input = np.deg2rad(phase_in_Degrees) #phase of the signal

    run = 0
    frequency_FFT_array = []
    FFT_found_ampl_array = []
    FFT_found_phase_array = []

    for run in range_run:
    
        guessA0 = max(exp_decay_mod_all_noise(x)) #max value is presumably the activity at t=0 (since it only decreases)
        guess_hLife_time = 365*24*60*60.0  #since all sources have a half life time in order of years, start with guess 1 yr
        guessExp = (guessA0, guess_hLife_time)
    
        found_fit_signal = fit_signal(exp_decay_mod_all_noise(x), guessExp, x)
        found_residual = residual(x, exp_decay_mod_all_noise(x), found_fit_signal)
        found_FFT_residual = FFT_residual (found_residual)
        found_FFTfreq = FFTfreq (found_residual)
    
    #print the found FFT frequency, amplitude and phase
        FFTarray = np.abs(found_FFT_residual/(len(found_FFT_residual)))
        max_index = np.argmax(FFTarray)
        max_value = np.max(FFTarray)
        frequency_FFT = np.abs(found_FFTfreq[max_index])/T # devide by T, because above it was per T seconds, 
                                                           # because we look at 1 sample per time T
        
        frequency_FFT_array.append(frequency_FFT)
        FFT_found_ampl = max_value
        FFT_found_ampl_array.append(FFT_found_ampl) 
        FFT_found_phase = np.abs(np.angle(found_FFT_residual[max_index]))
        FFT_found_phase_array.append(FFT_found_phase)

    mean_ampl_FFT_array = np.mean(FFT_found_ampl_array)
    FFT_found_ampl_varying_ampl_array.append(mean_ampl_FFT_array)
    stdv_ampl_FFT_array = np.std(FFT_found_ampl_array)
    stdv_FFT_found_ampl_varying_ampl_array.append(stdv_ampl_FFT_array)
    mean_freq_FFT_array = np.mean(frequency_FFT_array)
    frequency_FFT_varying_ampl_array.append(mean_freq_FFT_array)
    stdv_freq_FFT_array = np.std(frequency_FFT_array)
    stdv_frequency_FFT_varying_ampl_array.append(stdv_freq_FFT_array)
    mean_phase_FFT_array = np.mean(FFT_found_phase_array)
    FFT_found_phase_varying_ampl_array.append(mean_phase_FFT_array)
    stdv_phase_FFT_array = np.std(FFT_found_phase_array)
    stdv_FFT_found_phase_varying_ampl_array.append(stdv_phase_FFT_array)
                                           
np.savetxt('input_amplitude_array_0.0002_0.001', (ampl_input_array))
           
np.savetxt('mean_found_amplitude_array_50_0.0002_0.001', (FFT_found_ampl_varying_ampl_array))
np.savetxt('stdv_found_amplitude_array_50_0.0002_0.001', (stdv_FFT_found_ampl_varying_ampl_array))
np.savetxt('mean_found_frequency_array_50_0.0002_0.001', (frequency_FFT_varying_ampl_array))
np.savetxt('stdv_found_frequency_array_50_0.0002_0.001', (stdv_frequency_FFT_varying_ampl_array))
np.savetxt('mean_found_phase_array_50_0.0002_0.001', (FFT_found_phase_varying_ampl_array))
np.savetxt('stdv_found_phase_array_50_0.0002_0.001', (stdv_FFT_found_phase_varying_ampl_array))
